In [1]:
import pandas as pd
import os

# Définir l'affichage de toutes les colonnes
pd.set_option('display.max_columns', None)

In [41]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))

In [42]:
fonciere_data = pd.read_csv(parent_directory+"/data/fonciere.csv",sep=";")
fonciere_data['LIBCOM_treat'] = fonciere_data['LIBCOM'].apply(retraitement_str)

Liste des acronymes  

**CA** : Communauté d’agglomération  
**CC** : Communauté de communes  
**CU** : Communauté urbaine  
**MET** : Métropole  
**TEOM** : Taxe d’enlèvement des ordures ménagères  
**TFB** : Taxe foncière sur les propriétés bâties  
**TFNB** : Taxe foncière sur les propriétés non bâties  
**TH** : Taxe d’habitation  
**THRS** : Taxe d’habitation sur les résidences secondaires  
**THLV** : Taxe d’habitation sur les logements vacants  

In [43]:
colonnes_etude = ["EXERCICE","REG","LIBREG","DEP","LIBDEP","COM","LIBCOM","Taux_Global_TFNB","Taux_Global_TFB"]

In [44]:
fonciere_data.loc[(fonciere_data["EXERCICE"]==2023),colonnes_etude].head()

,EXERCICE,REG,LIBREG,DEP,LIBDEP,COM,LIBCOM,Taux_Global_TFNB,Taux_Global_TFB
1131,2023,44,GRAND-EST,55,MEUSE,113,CHEPPY,71.06,43.20
1132,2023,44,GRAND-EST,55,MEUSE,133,COUVERTPUIS,86.67,43.78
1133,2023,44,GRAND-EST,55,MEUSE,143,DAMLOUP,89.61,50.34
1134,2023,44,GRAND-EST,55,MEUSE,163,DONCOURT AUX TEMPLIERS,83.07,45.23
1135,2023,44,GRAND-EST,55,MEUSE,168,DUZEY,75.37,47.48


In [6]:
fonciere_data.loc[(fonciere_data["DEP"]=="75")&(fonciere_data["EXERCICE"]==2023),colonnes_etude]

,EXERCICE,REG,LIBREG,DEP,LIBDEP,COM,LIBCOM,Taux_Global_TFNB,Taux_Global_TFB
21143,2023,11,ILE-DE-FRANCE,75,PARIS,56,VILLE DE PARIS,28.86,21.15


In [19]:
import unicodedata
import re

def retraitement_str(text:str)->str:
    # minuscule
    text = text.lower()
    # remplacer les caractères accentués
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    # supprimer ponctuation
    text = re.sub(r'[^\w]', '', text)
    
    return text

print(retraitement_str(" le Pétit-quevillyç!") == "lepetitquevillyc")

True


In [17]:
expected = fonciere_data[fonciere_data.index==21143]
fonciere_data[fonciere_data.index==21143]

,EXERCICE,REG,LIBREG,DEP,LIBDEP,COM,LIBCOM,SIREPCI,OPTEPCI,Q03,MPOID,B12VOTE,B22,B32VOTE,B12TAFNB,B32TAFNB,B32MGPTAFNB,B52,B52A,B52gGEMAPI,Taux_Global_TFNB,E12VOTE,E22,E32VOTE,E52,E52A,E52TASA,E52gGEMAPI,Taux_Global_TFB,Taux_Plein_TEOM,H12VOTE,H22,H32VOTE,H52,H52A,H52gGEMAPI,Taux_Global_TH,IND_MAJOTHRS,THSURTAXRSTAU,INSEE COM,LIBCOM_treat
21143,2023,11,ILE-DE-FRANCE,75,PARIS,56,VILLE DE PARIS,200054781.0,FPU,Métropole du Grand Paris,2162598.0,25.31,0.0,0.0,0.0,0.0,2.31,0.59,0.493,0.155,28.86,20.5,0.0,0.0,0.211,0.176,0.193,0.072,21.15,6.21,20.32,0.0,0.0,0.16,0.17,0.0962,20.75,OUI,60.0,75056,villedeparis


In [22]:
def find_line_in_fonciere_df(dep:str,ville:str,data:pd.DataFrame=fonciere_data,exercice:int=2023,libcom:str="LIBCOM_treat")->pd.DataFrame:
    return data.loc[(data['DEP']==dep)&(data['EXERCICE']==exercice)&(data['LIBCOM_treat'].str.contains(retraitement_str(ville)))]

# assert find_line_in_fonciere_df('75','paRis') == expected
print(find_line_in_fonciere_df('75','p a R i s').equals(expected))


True


In [24]:
find_line_in_fonciere_df('76','petit quevilly')

,EXERCICE,REG,LIBREG,DEP,LIBDEP,COM,LIBCOM,SIREPCI,OPTEPCI,Q03,MPOID,B12VOTE,B22,B32VOTE,B12TAFNB,B32TAFNB,B32MGPTAFNB,B52,B52A,B52gGEMAPI,Taux_Global_TFNB,E12VOTE,E22,E32VOTE,E52,E52A,E52TASA,E52gGEMAPI,Taux_Global_TFB,Taux_Plein_TEOM,H12VOTE,H22,H32VOTE,H52,H52A,H52gGEMAPI,Taux_Global_TH,IND_MAJOTHRS,THSURTAXRSTAU,INSEE COM,LIBCOM_treat
85438,2023,28,NORMANDIE,76,SEINE-MARITIME,498,LE PETIT-QUEVILLY,200023414.0,FPU,Métropole Rouen Normandie,22283.0,68.56,0.0,2.6,0.0,37.0,0.0,0.173,0.0,0.0,108.33,58.58,0.0,0.0,0.111,0.0,0.0,0.0,58.69,8.06,18.42,0.0,8.35,0.0579,0.0,0.0,26.83,NON,NaN,76498,lepetitquevilly


Formule de calcul de la taxe fonciere

Taxe foncière = (Valeur locative cadastrale×50%) × Taux d’imposition

ValeurLocativeCadastrale = Surface pondérée x Tarif au m² x Coefficient de localisation [0,7-1,3] x Coefficient d'entretien [0,9-1,1]

Le coefficient de localisation peut varier de 0,7 à 1,3 selon la situation du bien dans la commune. Le coefficient d'entretien, quant à lui, oscille généralement entre 0,9 et 1,1 en fonction de l'état du bâtiment.

In [27]:
def taxe_fonciere(valeur_locative_cadastrale:float,taux_taxe_fonciere:float)->float:
    return valeur_locative_cadastrale*12*0.5*taux_taxe_fonciere

def valeur_locative_cadastrale(surface_ponderee:float,tarif_m2:float,coeff_localisation:float,coeff_entretien:float)->float:
    return surface_ponderee*tarif_m2*coeff_localisation*coeff_entretien


In [28]:
vlc = valeur_locative_cadastrale(65,20.3,0.95,0.9)
taxe_fonciere(vlc,0.2115)

1431.6509025

In [53]:
prefix = parent_directory+"/data/"
indicateurs_loyer = {e[:-4]:pd.read_csv(prefix+e,sep=";",encoding="latin-1") for e in os.listdir(prefix) if 'loy' in e}
for k in indicateurs_loyer.keys():
    indicateurs_loyer[k]['LIBGEO_treat'] = indicateurs_loyer[k]['LIBGEO'].apply(retraitement_str)

In [54]:
indicateurs_loyer.keys()

dict_keys(['loymai_92m2', 'loyapp_52m2', 'loyapp3_72m2', 'loyapp12_37m2'])

In [62]:
def select_dfname_by_surface(surface:float)->str:
    if surface >= 87:
        return 'loymai_92m2'
    elif surface >= 62:
        return 'loyapp3_72m2'
    elif surface >= 45:
        return 'loyapp_52m2'
    else:
        return 'loyapp12_37m2'

print(select_dfname_by_surface(16))
print(select_dfname_by_surface(46))
print(select_dfname_by_surface(80))
print(select_dfname_by_surface(116))

loyapp12_37m2
loyapp_52m2
loyapp3_72m2
loymai_92m2


In [52]:
indicateurs_loyer['loyapp_52m2']

,id_zone,INSEE_C,LIBGEO,EPCI,DEP,REG,loypredm2,lwr.IPm2,upr.IPm2,TYPPRED,nbobs_com,nbobs_mail,R2_adj,LIBGEO_treat
0,1,86051,Champagné-le-Sec,200070035,86,75,"8,48108601820233","6,60248185247569","10,8942094284101",maille,0,1044,"0,719908446134695",champagnelesec
1,1,16136,La Faye,200043016,16,75,"8,48108601820233","6,60248185247569","10,8942094284101",maille,4,1044,"0,719908446134695",lafaye
2,1,16002,Les Adjots,200043016,16,75,"8,48108601820233","6,60248185247569","10,8942094284101",maille,2,1044,"0,719908446134695",lesadjots
3,1,79150,Limalonges,200069755,79,75,"8,48108601820233","6,60248185247569","10,8942094284101",maille,3,1044,"0,719908446134695",limalonges
4,1,16104,Condac,200043016,16,75,"8,48108601820233","6,60248185247569","10,8942094284101",maille,11,1044,"0,719908446134695",condac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34955,DOM_9,97304,Kourou,200027548,973,3,"15,9611624882161","12,4535990029436","20,4566332925142",commune,759,773,"0,737270682644529",kourou
34956,DOM_9,97362,Papaichton,249730037,973,3,"13,3164044517987","10,3268119743639","17,1714782804308",maille,1,773,"0,737270682644529",papaichton
34957,DOM_9,97358,Saint-Élie,200027548,973,3,"13,3164044517987","10,3268119743639","17,1714782804308",maille,0,773,"0,737270682644529",saintelie
34958,DOM_9,97353,Maripasoula,249730037,973,3,"13,3164044517987","10,3268119743639","17,1714782804308",maille,13,773,"0,737270682644529",maripasoula


In [58]:
loyapp_52m2.loc[loyapp_52m2["LIBGEO"].str.contains('Paris 20')]

,id_zone,INSEE_C,LIBGEO,EPCI,DEP,REG,loypredm2,lwr.IPm2,upr.IPm2,TYPPRED,nbobs_com,nbobs_mail,R2_adj
19703,2784,75120,Paris 20e Arrondissement,200054781,75,11,"28,2934969807634","22,7823838781976","35,1377615125937",commune,14701,14701,"0,898130318666813"


In [45]:
float(loyapp_52m2.loc[loyapp_52m2['INSEE_C']=="75120","loypredm2"].values[0].replace(",","."))*0.8

22.634797584610723

In [55]:
float(fonciere_data.loc[(fonciere_data['LIBCOM'].str.contains("VILLE DE Paris".upper())) & (fonciere_data["EXERCICE"]==2023),"Taux_Global_TFB"].values[0])*0.01

0.2115